In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
from skimage import io, transform
import PIL
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

trainset = torchvision.datasets.CIFAR10(root='./image_files', train=True,
                                        download=False, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./image_files', train=False,
                                       download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [3]:
def imshow(img):
    unnormalized_image = img / 2 + 0.5     # unnormalize
    npimg = unnormalized_image.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        start_size = 32
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, start_size, 3, padding=(1,1))
        self.bn1 = nn.BatchNorm2d(start_size)
        self.conv2 = nn.Conv2d(start_size, start_size, 3, padding=(1,1))
        self.bn2 = nn.BatchNorm2d(start_size)
        self.proj2 = nn.Conv2d(start_size, start_size*2, 1)
        self.conv3 = nn.Conv2d(start_size*2, start_size*2, 3, padding=(1,1))
        self.bn3 = nn.BatchNorm2d(start_size*2)
        self.conv4 = nn.Conv2d(start_size*2, start_size*2, 3, padding=(1,1))
        self.bn4 = nn.BatchNorm2d(start_size*2)
        self.proj4 = nn.Conv2d(start_size*2, start_size*4, 1)
        self.conv5 = nn.Conv2d(start_size*4, start_size*4, 3, padding=(1,1))
        self.bn5 = nn.BatchNorm2d(start_size*4)
        self.conv6 = nn.Conv2d(start_size*4, start_size*4, 3, padding=(1,1))
        self.bn6 = nn.BatchNorm2d(start_size*4)
        self.proj6 = nn.Conv2d(start_size*4, start_size*8, 1)
        self.conv7 = nn.Conv2d(start_size*8, start_size*8, 3, padding=(1,1))
        self.bn7 = nn.BatchNorm2d(start_size*8)
        self.conv8 = nn.Conv2d(start_size*8, start_size*8, 3, padding=(1,1))
        self.bn8 = nn.BatchNorm2d(start_size*8)
        self.fc1 = nn.Linear(start_size*8, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.proj2(self.pool(F.relu(self.bn2(x + self.conv2(x)))))
        x = F.relu(self.bn3(x + self.conv3(x)))
        x = self.proj4(self.pool(F.relu(self.bn4(x + self.conv4(x)))))
        x = F.relu(self.bn5(x + self.conv5(x)))
        x = self.proj6(self.pool(F.relu(self.bn6(x + self.conv6(x)))))
        x = F.relu(self.bn7(x + self.conv7(x)))
        x = self.pool(F.relu(self.bn8(x + self.conv8(x))))
        x = F.avg_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [5]:
net = Net().cuda()
criterion = nn.CrossEntropyLoss()


In [6]:
for epoch in range(20):  # loop over the dataset multiple times
    lr = 1e-2 * (1.25 ** (-epoch))
    print('Epoch {0}, learning rate {1}'.format(epoch + 1, lr))
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs_cpu, labels_cpu = data
        inputs, labels = inputs_cpu.cuda(), labels_cpu.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


Epoch 1, learning rate 0.01
[1,   100] loss: 1.851
[1,   200] loss: 1.519
[1,   300] loss: 1.423
[1,   400] loss: 1.282
[1,   500] loss: 1.225
[1,   600] loss: 1.118
[1,   700] loss: 1.087
Epoch 2, learning rate 0.008
[2,   100] loss: 0.955
[2,   200] loss: 0.910
[2,   300] loss: 0.897
[2,   400] loss: 0.869
[2,   500] loss: 0.845
[2,   600] loss: 0.820
[2,   700] loss: 0.800
Epoch 3, learning rate 0.0064
[3,   100] loss: 0.728
[3,   200] loss: 0.731
[3,   300] loss: 0.701
[3,   400] loss: 0.706
[3,   500] loss: 0.685
[3,   600] loss: 0.688
[3,   700] loss: 0.689
Epoch 4, learning rate 0.00512
[4,   100] loss: 0.609
[4,   200] loss: 0.642
[4,   300] loss: 0.597
[4,   400] loss: 0.643
[4,   500] loss: 0.614
[4,   600] loss: 0.600
[4,   700] loss: 0.606
Epoch 5, learning rate 0.004096000000000001
[5,   100] loss: 0.557
[5,   200] loss: 0.559
[5,   300] loss: 0.574
[5,   400] loss: 0.571
[5,   500] loss: 0.545
[5,   600] loss: 0.548
[5,   700] loss: 0.547
Epoch 6, learning rate 0.00327680

In [7]:
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 85 %


In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 95 %
Accuracy of   car : 100 %
Accuracy of  bird : 78 %
Accuracy of   cat : 69 %
Accuracy of  deer : 85 %
Accuracy of   dog : 73 %
Accuracy of  frog : 77 %
Accuracy of horse : 93 %
Accuracy of  ship : 92 %
Accuracy of truck : 100 %
